## Preparação

In [ ]:
# Atualiando o Plotly
!pip install -U plotly

In [ ]:
!pip install -q yfinance

In [49]:
import yfinance as yf
from datetime import date
import plotly.express as px

import pandas as pd
pd.options.plotting.backend = "plotly"

In [ ]:
def consulta_bc(codigo_serie, dataInicial='01/01/1900',  dataFinal=date.today().strftime('%d/%m/%Y')):  
  url = f'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={dataInicial}&dataFinal={dataFinal}'
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  return df

In [ ]:
data_inicial = '02/01/1995'
data_final   = '16/05/2021'

In [ ]:
ibov = yf.download('^BVSP', start=pd.to_datetime(data_inicial, dayfirst=True), end=pd.to_datetime(data_final, dayfirst=True))['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [ ]:
ibov

Date
1995-01-02      4301.000000
1995-01-03      4098.000000
1995-01-04      3967.899902
1995-01-05      4036.699951
1995-01-06      3827.399902
                  ...      
2021-05-10    121909.000000
2021-05-11    122964.000000
2021-05-12    119710.000000
2021-05-13    120706.000000
2021-05-14    121881.000000
Name: Adj Close, Length: 6519, dtype: float64

## IBOV Dolarizado

In [ ]:
dolar_ptax = consulta_bc(1, data_inicial, data_final)
dolar_ptax

In [ ]:
ibov_dolarizado = ibov / dolar_ptax['valor']

In [ ]:
ibov_dolarizado

1995-01-02     5089.940828
1995-01-03     4843.971631
1995-01-04     4690.189010
1995-01-05     4782.819847
1995-01-06     4551.010585
                  ...     
2021-05-10    23342.141038
2021-05-11    23465.068794
2021-05-12    22865.929364
2021-05-13    22854.492095
2021-05-14    23126.885638
Length: 6621, dtype: float64

In [ ]:
df_ibov = pd.concat([ibov, ibov_dolarizado], axis=1, keys=['IBOV', 'IBOV_USD']).dropna()

In [ ]:
df_ibov.plot()

## IBOV Declacionado pelo IPCA

In [ ]:
ipca = consulta_bc(433, data_inicial, data_final)

In [ ]:
ipca.columns = ['mensal']

In [ ]:
ipca['acumulado'] = (1 + (ipca['mensal'] / 100)).cumprod().shift(1)
ipca['acumulado'].iloc[0] = 1

In [ ]:
ipca

In [ ]:
(ibov / ipca['acumulado']).head(30)

1995-01-01            NaN
1995-01-02            NaN
1995-01-03            NaN
1995-01-04            NaN
1995-01-05            NaN
1995-01-06            NaN
1995-01-09            NaN
1995-01-10            NaN
1995-01-11            NaN
1995-01-12            NaN
1995-01-13            NaN
1995-01-16            NaN
1995-01-17            NaN
1995-01-18            NaN
1995-01-19            NaN
1995-01-20            NaN
1995-01-23            NaN
1995-01-24            NaN
1995-01-26            NaN
1995-01-27            NaN
1995-01-30            NaN
1995-01-31            NaN
1995-02-01    3923.107226
1995-02-02            NaN
1995-02-03            NaN
1995-02-06            NaN
1995-02-07            NaN
1995-02-08            NaN
1995-02-09            NaN
1995-02-10            NaN
dtype: float64

In [ ]:
join = ipca.join(ibov, how='outer').fillna(method='ffill').dropna()
join

In [ ]:
df_ibov['defla_ipca'] = (join['Adj Close'] / join['acumulado'])

In [ ]:
df_ibov.plot()

## Deflacionado pelo IGP-DI

In [ ]:
igpdi = consulta_bc(190, data_inicial, data_final)

In [ ]:
igpdi.columns = ['mensal']

In [ ]:
igpdi['acumulado'] = (1 + (igpdi['mensal'] / 100)).cumprod().shift(1)
igpdi['acumulado'].iloc[0] = 1

In [ ]:
igpdi

In [ ]:
join = igpdi.join(ibov, how='outer').fillna(method='ffill')

In [ ]:
df_ibov['defla_igpdi'] = (join['Adj Close'] / join['acumulado'])

In [ ]:
fig = df_ibov.plot()
fig.layout.autosize=False
fig.layout.xaxis.title = "Data"
fig.layout.yaxis.title = "Pontos"
fig.layout.title = "Índice Bovespa Dolarizado e Deflacionado"
fig.layout.legend.title = "Índice"


fig.show()